<a href="https://colab.research.google.com/github/jaya-shankar/DSS_project/blob/main/combine_covid_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setting up enviroment

In [ ]:
!sudo apt-get install git-lfs
!git lfs install --system --skip-repo
!pip install tensorflow_decision_forests
!pip install wurlitzer

In [ ]:
!rm -rf DSS_project

!git clone https://github.com/jaya-shankar/DSS_project.git

In [3]:
import pandas as pd
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split

In [4]:
root_path = "/content/DSS_project/datasets"
raw_path = "/content/DSS_project/raw_datasets"

## cleaning up us-counties dataset

In [ ]:
data_df = pd.read_csv(f"{raw_path}/us-counties.csv")
data_df

In [ ]:
p = data_df.groupby(by = ['date','state_code']).sum()
p.drop(columns=['fips'],inplace=True)
p = p.reset_index()
p

In [ ]:
states = set(p['state_code'])
states

In [93]:
updated_df = pd.DataFrame(columns = ['date','state_code','cases','deaths'])
for state in states:
  p_s = p[p.state_code==state]
  p_s['cases'] = p_s['cases'] - p_s['cases'].shift(1)
  p_s['deaths'] = p_s['deaths'] - p_s['deaths'].shift(1)
  updated_df = pd.concat([updated_df, p_s])
display(updated_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,state_code,cases,deaths
625,2020-03-13,PR,NaN,NaN
676,2020-03-14,PR,1.0,0.0
728,2020-03-15,PR,1.0,0.0
780,2020-03-16,PR,0.0,0.0
832,2020-03-17,PR,0.0,0.0
...,...,...,...,...
40531,2022-04-05,SC,1566.0,59.0
40584,2022-04-06,SC,0.0,0.0
40637,2022-04-07,SC,0.0,0.0
40690,2022-04-08,SC,0.0,0.0


In [94]:
updated_df.columns
updated_df.insert(0, 'state_code', updated_df.pop('state_code'))
updated_df.dropna(inplace=True)

In [104]:
updated_df.drop(updated_df[(updated_df['cases'] < 0) | (updated_df['deaths'] < 0)].index, inplace = True)
updated_df.sort_values(by=['date'], inplace=True)
updated_df.reset_index(drop=True, inplace=True)

In [106]:
updated_df

,state_code,date,cases,deaths
0,WA,2020-01-22,0.0,0.0
1,WA,2020-01-23,0.0,0.0
2,WA,2020-01-24,0.0,0.0
3,WA,2020-01-25,0.0,0.0
4,IL,2020-01-25,0.0,0.0
...,...,...,...,...
40406,NV,2022-04-09,174.0,10.0
40407,NY,2022-04-09,3263.0,7.0
40408,IN,2022-04-09,0.0,5.0
40409,GA,2022-04-09,0.0,0.0


In [107]:
updated_df['cases'].value_counts(bins=10, sort=False)

(-227.973, 22797.2]     40007
(22797.2, 45594.4]        267
(45594.4, 68391.6]         86
(68391.6, 91188.8]         29
(91188.8, 113986.0]         6
(113986.0, 136783.2]        8
(136783.2, 159580.4]        1
(159580.4, 182377.6]        3
(182377.6, 205174.8]        1
(205174.8, 227972.0]        3
Name: cases, dtype: int64

## combining us-counties and general df

In [ ]:
general_df = pd.read_csv('/content/DSS_project/datasets/combined_general_data.csv')
# general_df

In [ ]:
combined_df = pd.merge(updated_df, general_df, on="state_code")
# combined_df

In [116]:
combined_df_cpy = combined_df.copy()
combined_df_cpy.drop(columns=['state_code','date','deaths'],inplace=True)
combined_df_cpy['cases'].value_counts(bins=10, sort=False)

(-227.973, 22797.2]     40007
(22797.2, 45594.4]        267
(45594.4, 68391.6]         86
(68391.6, 91188.8]         29
(91188.8, 113986.0]         6
(113986.0, 136783.2]        8
(136783.2, 159580.4]        1
(159580.4, 182377.6]        3
(182377.6, 205174.8]        1
(205174.8, 227972.0]        3
Name: cases, dtype: int64

In [117]:
train_df, test_df = train_test_split(combined_df_cpy, test_size=0.3)
train_df = train_df.sample(frac = 0.05)
test_df = test_df.sample(frac = 0.05)
train_df

,cases,no_of_hospitals,miles_of_road,percent_democrat,population,avg_income,mean_travel_time,above_65,percent_white,persons_per_household,percent_in_poverty
17514,3141.0,114,80006.0,36.000000,9011672,47551.0,23.0,16.0,84.0,2.00,16.0
28579,4108.0,158,122040.0,51.000000,20154662,51071.0,24.0,17.0,74.0,2.00,14.0
28031,986.0,227,122991.0,45.947961,23598896,52758.0,23.5,17.1,78.7,2.43,13.9
27928,2806.0,227,122991.0,45.947961,23598896,52758.0,23.5,17.1,78.7,2.43,13.9
37919,566.0,58,19916.0,NaN,6571748,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2052,2123.0,397,175562.0,65.934760,79076446,57347.0,29.3,14.3,36.8,2.96,12.8
18312,328.0,26,4501.0,65.042567,2910542,47234.0,27.4,18.4,21.8,3.02,8.8
35076,1207.0,38,22851.0,55.000000,2724718,50516.0,24.0,20.0,93.0,2.00,11.0
31213,0.0,52,72092.0,55.400905,4235044,45637.0,22.1,17.5,37.1,2.64,19.5


In [118]:
# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="cases",task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="cases",task=tfdf.keras.Task.REGRESSION)

# Train a Random Forest model.
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.fit(train_ds)

# Summary of the model structure.
model.summary()

# Evaluate the model.
model.evaluate(test_ds)


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


Use /tmp/tmpeqlhgdu4 as temporary training directory
Starting reading the dataset
1/2 [==============>...............] - ETA: 0s
Dataset read in 0:00:00.176047
Training model
Model trained in 0:00:00.613671
Compiling model
2/2 [==============================] - 1s 704ms/step
Model: "random_forest_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (10):
	above_65
	avg_income
	mean_travel_time
	miles_of_road
	no_of_hospitals
	percent_democrat
	percent_in_poverty
	percent_white
	persons_per_household
	population

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.               "__LABEL"  7.524656 ################
    2.              "above_65"  6.556533 #############
    3.      "percent_democrat"  6.

0.0

In [74]:
data_df.to_csv("/content/DSS_project/datasets/covid_cases.csv")